# Week 10: Sentiment Experiment

There are other libraries than TextBlob that do sentiment analysis, so let's compare their results.  Another popular one is called Vader.

In [ ]:
carnivores = {
    "python": "A large heavy-bodied nonvenomous snake that kills poor prey by constriction and asphyxiation",
    "panda": "A large bearlike mammal that, while technically a carnivore, is in practice a vegetarian, eating only bamboo",
    "kitten": "A delightful, fuzzy creature whose natural prey is cat food (dry or wet) and, especially, treats"
}

In [ ]:
!pip install textblob
!pip install nltk

In [ ]:
from textblob import TextBlob
import nltk

blob = TextBlob(carnivores["panda"])
print(blob)
print(f"Polarity {blob.sentiment.polarity}")
print(f"Subjectivity {blob.sentiment.subjectivity}")

Alternative algorithm:

  > VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.
  [VADER readme](https://github.com/cjhutto/vaderSentiment/tree/master)

> [VADER aims to properly handle] sentences with:

> - typical negations (e.g., "*not* good")
- use of contractions as negations (e.g., "*wasn't* very good")
- conventional use of **punctuation** to signal increased sentiment intensity (e.g., "Good!!!")
- conventional use of **word-shape** to signal emphasis (e.g., using ALL CAPS for words/phrases)
- using **degree modifiers** to alter sentiment intensity (e.g., intensity *boosters* such as "very" and intensity *dampeners* such as "kind of")
- understanding many **sentiment-laden slang** words (e.g., 'sux')
- understanding many sentiment-laden **slang words as modifiers** such as 'uber' or 'friggin' or 'kinda'
- understanding many sentiment-laden **emoticons** such as :) and :D
- translating **utf-8 encoded emojis** such as 💘 and 💋 and 😁
- understanding sentiment-laden **initialisms and acronyms** (for example: 'lol')

Take a look at [VADER paper](https://ojs.aaai.org/index.php/ICWSM/article/view/14550/14399)

## So, let's set up our small investigation, comparing VADER and TextBlob approaches

1. Define (tricky) examples

In [ ]:
examples_bow = [
    "It was the best of times, it was the worst of times.",
    "I love how tragic her story is; it makes me feel alive.",
    "That poem wasn’t bad at all!",
    "The character’s demise was inevitable; simply tragic.",
    "Oh, fantastic... yet another twist ending.",
    "I couldn’t put the book down; it was haunting, to say the least.",
    "She was calm, almost too calm, like the eye of a storm.",
    "The protagonist had a truly unforgettable experience.",
    "Thank goodness it’s over. That was exhausting.",
    "This plot twist is simply too much... breathtaking!",
    "Wow, thanks for ruining my day with that spoiler! 😒",
    "Absolutely loved the movie... except for that ending, ugh!",
    "Can’t believe I waited hours for this. What a waste!",
    "OMG, this is the best thing I've seen all week!!!",
    "This product is seriously underrated, honestly amazing!",
    "LOL, yeah right, as if this would actually work... 🙄",
    "Finally finished it... mixed feelings, to be honest.",
    "You have to read this book—it’s like nothing else!",
    "Just what I needed... another delay. Fantastic. 🤦‍♂️",
    "I'm impressed! Didn’t expect it to be this good!"
]

print(len(examples_bow))


2. Formulate how the results should look like

In [ ]:
audit_results = {
    "Sentence": [],
    "VADER Sentiment": [],
    "VADER Score": [],
    "TextBlob Sentiment": [],
    "TextBlob Polarity": [],
    "Difference Detected": []
}

3. Set up VADER and TextBlob

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

# Initialize VADER sentiment analyzer
vader_analyzer = SentimentIntensityAnalyzer()
print(vader_analyzer.polarity_scores(carnivores["panda"]))
print(vader_analyzer.polarity_scores(carnivores["python"]))
print(vader_analyzer.polarity_scores(carnivores["kitten"]))

In [ ]:
print(vader_analyzer.polarity_scores(carnivores["kitten"])['compound'])

4. Analyze sentiment and write down the results

In [ ]:
for sentence in examples_bow:
    # VADER sentiment
    vader_scores = vader_analyzer.polarity_scores(sentence)
    vader_sentiment = "Positive" if vader_scores['compound'] >= 0.05 else "Negative" if vader_scores['compound'] <= -0.05 else "Neutral"


    # TextBlob sentiment
    blob = TextBlob(sentence)
    blob_polarity = blob.sentiment.polarity
    blob_subjectivity = blob.sentiment.subjectivity
    blob_sentiment = "Positive" if blob_polarity > 0 else "Negative" if blob_polarity < 0 else "Neutral"

    # Detect if there is a difference in sentiment
    difference_detected = vader_sentiment != blob_sentiment

    # Append results to the data dictionary
    audit_results["Sentence"].append(sentence)
    audit_results["VADER Sentiment"].append(vader_sentiment)
    audit_results["VADER Score"].append(vader_scores['compound'])
    audit_results["TextBlob Sentiment"].append(blob_sentiment)
    audit_results["TextBlob Polarity"].append(blob_polarity)
    audit_results["Difference Detected"].append(difference_detected)

5. Transform results in analysis-friendly form

Our friend Pandas allows us to quite easily make a new DataFrame out of this kind of data, with its `pd.DataFrame()` method.

The `pd.DataFrame()` method takes as its argument... **a dictionary**! (See why we had to finally learn about dictionaries??). It expects this argument to be formatted as follows:

```
new_df = pd.DataFrame(
    {
        'column1': list1,
        'column2': list2,
        'column3': list3
    }
)
```

Of course, you could also write this same command without all the tabs and newlines as follows:

`new_df = pd.DataFrame({'column1': list1, 'column2': list2, 'column3': list3})`

In [ ]:
# Create a DataFrame from dictionary with column names as keys and column data as lists of values
import pandas as pd
df = pd.DataFrame(audit_results)
df

6. Look at the differences

In [ ]:
df[df["Difference Detected"] == True]
## or just
#df[df["Difference Detected"]
## why?


In [ ]:
len(df[df["Difference Detected"] == True]) / len(df)

In [ ]:
pd.crosstab(df["VADER Sentiment"], df["TextBlob Sentiment"])

- What do we think about the results?
- Key question: How typical they are for tasks at hand? What other troubles we may encounter?
- What extra questions should we ask ourselves?